# OutputFixingParser

`OutputFixingParser`는 출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공합니다. 이 파서는 기본적으로 다른 파서, 예를 들어 `PydanticOutputParser` 를 래핑하고, 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정하도록 설계되었습니다.

이러한 접근 방식의 핵심은, 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나올 경우, `OutputFixingParser` 가 자동으로 형식이 잘못된 출력을 인식하고, 이를 수정하기 위한 새로운 명령어와 함께 모델에 다시 제출한다는 것입니다. 이 과정에서, 수정을 위한 명령어는 오류를 정확히 지적하고, 올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 합니다.

예를 들어, `PydanticOutputParser` 를 사용하여 특정 데이터 스키마를 준수하는 출력을 생성하려고 했지만, 일부 필드가 누락되었거나 데이터 유형이 잘못된 경우가 발생할 수 있습니다. 이때 `OutputFixingParser` 는 다음 단계로, 해당 오류를 수정하는 지시를 포함한 새로운 요청을 LLM에 제출합니다. LLM은 이 지시를 바탕으로 오류를 수정한 새로운 출력을 생성하게 됩니다.


In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [2]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Actor(BaseModel):
    name: str = Field(description='name of an actor')
    film_names: List[str] = Field(description="list of names of films they starred in")

actor_query = "Generate the filmography for a random actor."
parser = PydanticOutputParser(pydantic_object=Actor)

In [4]:
misformatted = "{'name':'Tom Hanks', 'film_names':['Forrest Gump']}"

parser.parse(misformatted)

OutputParserException: Invalid json output: {'name':'Tom Hanks', 'film_names':['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [6]:
from langchain.output_parsers import OutputFixingParser
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [7]:
misformatted

"{'name':'Tom Hanks', 'film_names':['Forrest Gump']}"

In [8]:
actor = new_parser.parse(misformatted)

In [9]:
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])